In [1]:
import os
import numpy as np

from pydub import AudioSegment
from pydub.utils import mediainfo

#settings
import config

## Reading recordings

In [2]:
all_s=[]
all_label=[]
all_id=[]
all_fs=[]

In [3]:
#Read wav data set

if config.featExtr_skip is False:
    print("Readings wavs...")

    #only list files in FOLDER_PATH directory
    wav_files = [f for f in os.listdir(config.FOLDER_PATH) if os.path.isfile(os.path.join(config.FOLDER_PATH, f))]
    for file_name in wav_files:
    
        fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
        #full path file name
        full_fname = config.FOLDER_PATH+file_name
        print(full_fname)
    
        # load audio
        s = AudioSegment.from_wav(full_fname)
        print(full_fname)
        all_s.append(s)
        #sampling rate:
        info = mediainfo(full_fname)
        fs = float(info['sample_rate'])
        all_fs.append(fs)
    
        #get ID of recording
        ID = fname_noExt.split('-')[-2] #for the current type of naming
        #print(file_name)
        #print(ID)
        all_id.append(ID)
    
        #get label
        label = fname_noExt.split('-')[-1] #for the current type of naming
        #print(label)
        all_label.append(label)

Readings wavs...
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 51-LkxvBb2VXbs-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 51-LkxvBb2VXbs-Dry.wav
data/YT_set/edited_wavs/edit_Wet coughing-0QQxKN-KC1U-Wet.wav
data/YT_set/edited_wavs/edit_Wet coughing-0QQxKN-KC1U-Wet.wav
data/YT_set/edited_wavs/edit_Dry Coughing Fit in the Afternoon.-A5s2ZgwQ1VM-Dry.wav
data/YT_set/edited_wavs/edit_Dry Coughing Fit in the Afternoon.-A5s2ZgwQ1VM-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 77-2Mw-s5jnqXU-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 77-2Mw-s5jnqXU-Wet.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 46-dg-I9j76-t8-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 46-dg-I9j76-t8-Wet.wav
data/YT_set/edited_

Listening to some of the audios

In [4]:
np.where(np.array(all_label)=='Dry')

(array([ 0,  1,  3,  7,  8,  9, 10, 15, 16, 17, 19, 20, 23, 27, 31]),)

In [5]:
np.where(np.array(all_label)=='Wet')

(array([ 2,  4,  5,  6, 11, 12, 13, 14, 18, 21, 22, 24, 25, 26, 28, 29, 30,
        32, 33, 34, 35]),)

In [6]:
s=all_s[15]
s

## Feature extraction

In [7]:
import featureExtractionFunctions as featExtrLib

if config.featExtr_skip is False:

    feats = featExtrLib.feature_extraction_Step(all_s,all_id,all_label)
    
       

High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass 

## Load  (or store) features 

In [8]:
feats_fname = 'feats_df.pkl'

if config.featExtr_skip is False:
    #Store feature df
    feats.to_pickle(feats_fname)
else:
    #Load feature df
    feats = pd.read_pickle(feats_fname)

## Pre-processing of features

In [9]:
feats_df = featExtrLib.processingNaNvalues(feats)

In [10]:
label_dict = featExtrLib.createLabelDict_addLabel2df(feats_df)
mean_std_feats = featExtrLib.frame_mean_std_chunk_modeling (feats_df,label_dict,10)

## Model training + evaluation (average on 20 runs)

In [11]:
X_train = mean_std_feats.drop(['label','Id','subIdx'], 1).copy()
y_train =  mean_std_feats['label'].copy()

ID_train = mean_std_feats['Id']

#ID_train.size

In [12]:
import classifEvaluationFunctions as evalLib

nr_runs = 20
scores_list = ['accuracy','recall','f1','precision']
avg_scores = evalLib.getAvgEvalScores_ofTrainRuns(nr_runs,X_train,y_train,ID_train,label_dict, scores_list, True)

Run 0:

accuracy     0.555556
recall       0.533333
f1           0.532468
precision    0.535117


Run 1:

accuracy     0.638889
recall       0.623810
f1           0.624699
precision    0.626623


Run 2:

accuracy     0.722222
recall       0.685714
f1           0.687500
precision    0.740741


Run 3:

accuracy     0.611111
recall       0.600000
f1           0.600000
precision    0.600000


Run 4:

accuracy     0.611111
recall       0.590476
f1           0.590909
precision    0.595318


Run 5:

accuracy     0.694444
recall       0.671429
f1           0.674074
precision    0.687500


Run 6:

accuracy     0.666667
recall       0.657143
f1           0.657143
precision    0.657143


Run 7:

accuracy     0.750000
recall       0.738095
f1           0.740176
precision    0.743506


Run 8:

accuracy     0.638889
recall       0.614286
f1           0.614815
precision    0.625000


Run 9:

accuracy     0.611111
recall       0.590476
f1           0.590909
precision    0.595318


Run 10:

accuracy   

In [13]:
#Print average values:
print('Average scores:')
evalLib.print_scores(avg_scores)

Average scores:
accuracy     0.654167
recall       0.635476
f1           0.635648
precision    0.644348
